## Discovering matrix multiplication algorithms

Here we illustrate how cool FedroTensor is at finding fast matrix multiplication algorithms.
Namely, our task is to minimise the number of scalar multiplications.
This problem is equivalent to finding the lowest possible CP rank of a certain 3-dimensional tensor.

Let's start with the 2x2 case. The naive approach results in 8 (=2x2x2) scalar multiplications, but there is a way to do it with only 7 multiplications, known as the Strassen algorithm.
Let's see if our methods can recover it.

In [1]:
from FedroTensor.tensor_rank import cp_rank
from FedroTensor.matmul_tensor import build_matmul_tensor

T = build_matmul_tensor(2, 2, 2)
r, factors = cp_rank(T)
r

7

The rank indeed corresponds to the Strassen algorithm cost. Let's see if we can recover the algorithm from the factors.

In [2]:
factors

[array([[-0.91604053, -1.04797915,  0.04798972, -0.97228425, -0.04033627,
         -0.46841578, -0.51135352],
        [ 0.02308362,  0.02618851,  0.07641874,  0.10049277,  0.08055061,
          0.93541443, -0.81567343],
        [ 0.64154406, -0.47116093,  0.62031795,  0.15212342, -0.52260128,
         -0.21039562,  0.35803647],
        [-0.01605648,  0.01178611,  0.98923043,  0.98506236,  1.04312642,
          0.4203476 ,  0.57119605]]),
 array([[-0.784294  , -0.14594683, -0.96202094, -0.92970176,  0.90084842,
         -0.00412017, -0.02188905],
        [ 0.64320303, -1.01814861, -0.36283573, -0.37720918,  0.33994756,
         -0.0280733 ,  0.01798729],
        [ 0.49180259, -0.07291947, -0.48186464,  0.41766118, -0.56497485,
         -0.1588752 , -0.87011186],
        [-0.40316805, -0.50980325, -0.18170665, -0.91362354, -0.21286316,
         -1.10911864,  0.71337682]]),
 array([[ 0.16880889, -0.23046733, -0.65430541,  0.97431601,  0.30126016,
         -0.7425601 ,  1.17795876],
      

Not helpful indeed...

Ideally, we would like to have integer factors, or at least the rational ones. We can compute them by specifying `rational=True`:

In [3]:
r, factors = cp_rank(T, rational=True)
factors

[array([[ 0.,  1., -1.,  0.,  0., -1., -1.],
        [ 0.,  0.,  1.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0., -1.,  0.],
        [-1.,  0.,  0., -1.,  1.,  0., -1.]]),
 array([[-1.,  0.,  0., -1.,  0.,  1., -1.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [-1.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0., -1., -1.,  0., -1.,  0.,  1.]]),
 array([[-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [ 0., -1., -1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  1.,  0., -1., -1.]])]

That's much better, as we can now write down the algorithm precisely.

Let's consider the 3x3 case. The best known algorithm (the Laderman's algorithm) has 23 multiplications. Remarkably, our method manages to find rank 23:

In [4]:
T = build_matmul_tensor(3, 3, 3)
r, factors = cp_rank(T)
r

23

Computing rational factors takes slightly longer, but it's definitely worth it:

In [5]:
r, factors = cp_rank(T, rational=True)
factors

[array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         -1., -1., -1.,  0., -1., -1.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., -1.,  0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0., -1.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
          1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
          1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

Finally, it's time for a benchmark on different sizes (n, m, p).

In [6]:
sizes = [(2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 2, 5), (2, 3, 3), (2, 3, 4), (2, 3, 5), (2, 4, 4), (2, 4, 5), (2, 5, 5), (3, 3, 3), (3, 3, 4), (3, 3, 5), (3, 4, 4), (3, 4, 5), (4, 4, 4)]
true_rs = [7, 11, 14, 18, 15, 20, 25, 26, 33, 40, 23, 29, 36, 38, 47, 49]
print('n\tm\tp\tr\ttrue_r')
for (n, m, p), true_r in zip(sizes, true_rs):
    T = build_matmul_tensor(n, m, p)
    r, _ = cp_rank(T)
    print(f'{n}\t{m}\t{p}\t{r}\t{true_r}')

n	m	p	r	true_r
2	2	2	7	7
2	2	3	11	11
2	2	4	14	14
2	2	5	18	18
2	3	3	15	15
2	3	4	20	20
2	3	5	25	25
2	4	4	26	26
2	4	5	33	33
2	5	5	43	40
3	3	3	23	23
3	3	4	31	29
3	3	5	38	36
3	4	4	41	38
3	4	5	51	47
4	4	4	55	49


Notice our method finds optimal ranks when the rank is less than 25. In the larger cases, it's slightly worse (for now). One can increase `num_attempts` from 10 to, say, 100, but then the computation becomes proportionally slower.